# Parte 2: Extracción de Descriptores Clásicos

**Taller 3 - Clasificación de Imágenes Médicas**

---

## Objetivos

### A. Descriptores de Forma (mínimo 3)
1. **HOG** (Histogram of Oriented Gradients)
2. **Momentos de Hu** (7 momentos invariantes)
3. **Descriptores de Contorno** (área, perímetro, circularidad, excentricidad)
4. **Fourier Shape Descriptors**

### B. Descriptores de Textura (mínimo 3)
1. **LBP** (Local Binary Patterns)
2. **GLCM** (Gray Level Co-occurrence Matrix)
3. **Filtros de Gabor**
4. **Estadísticas de primer orden**


## 1. Configuración e Imports


In [ ]:
# Imports estándar
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Agregar src al path
sys.path.insert(0, os.path.abspath('..'))

# Imports de nuestros módulos
from src.data_loader import load_image_paths, split_by_set
from src.preprocessing import read_and_preprocess, IMG_SIZE

# Descriptores de forma
from src.descriptors.shape import (
    descriptor_hog,
    descriptor_hu,
    descriptor_contorno,
    descriptor_fourier,
    segmentar,
    test_hu_invariances
)

# Descriptores de textura
from src.descriptors.texture import (
    descriptor_lbp,
    descriptor_glcm,
    descriptor_gabor
)

# Estadísticas
from src.descriptors.statistics import first_order_stats

# Configuración
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Imports completados correctamente ✓")


In [ ]:
# Cargar datos
DATA_DIR = "../data/chest_xray/chest_xray"
paths, labels = load_image_paths(DATA_DIR)
(paths_train, labels_train), (paths_val, labels_val), (paths_test, labels_test) = split_by_set(paths, labels)

# Seleccionar imagen de muestra para demostraciones
sample_idx = np.random.choice(len(paths_train))
sample_path = paths_train[sample_idx]
sample_label = labels_train[sample_idx]

# Cargar y preprocesar imagen de muestra
img = read_and_preprocess(sample_path, apply_clahe=True)

print(f"Imagen de muestra: {sample_label}")
print(f"Shape: {img.shape}")
print(f"Rango: [{img.min():.3f}, {img.max():.3f}]")

# Visualizar
plt.figure(figsize=(6, 6))
plt.imshow(img, cmap='gray')
plt.title(f'Imagen de Muestra: {sample_label}')
plt.axis('off')
plt.show()


---
# A. DESCRIPTORES DE FORMA

## 2. HOG (Histogram of Oriented Gradients)

HOG captura información sobre la distribución de gradientes en la imagen, útil para detectar bordes y formas.

### Parámetros a explorar:
- **pixels_per_cell**: Tamaño de cada celda (8×8, 16×16, 32×32)
- **orientations**: Número de bins de orientación (6, 9, 12)


In [ ]:
# Calcular HOG con parámetros por defecto
hog_vec, hog_img = descriptor_hog(img, pixels_per_cell=(16, 16), orientations=9)

print(f"Vector HOG shape: {hog_vec.shape}")
print(f"Dimensionalidad del descriptor: {len(hog_vec)} características")

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(img, cmap='gray')
axes[0].set_title('Imagen Original')
axes[0].axis('off')

axes[1].imshow(hog_img, cmap='inferno')
axes[1].set_title('Visualización HOG\n(pixels_per_cell=16, orientations=9)')
axes[1].axis('off')

plt.tight_layout()
plt.savefig('../results/02_hog_basic.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Explorar variaciones de parámetros HOG
bins_list = [12, 9, 6]
cells_list = [(8, 8), (16, 16), (32, 32)]

fig, axs = plt.subplots(3, 3, figsize=(14, 14))
fig.suptitle("Variaciones del Descriptor HOG", fontsize=16, y=1.02)

# Fila 1: Variar orientaciones (bins) con celda fija 16x16
for i, bins in enumerate(bins_list):
    _, vis = descriptor_hog(img, pixels_per_cell=(16, 16), orientations=bins)
    axs[0, i].imshow(vis, cmap='inferno')
    axs[0, i].set_title(f'cell=(16,16), bins={bins}')
    axs[0, i].axis('off')

# Fila 2: Variar tamaño de celda con bins fijos en 9
for i, cell in enumerate(cells_list):
    vec, vis = descriptor_hog(img, pixels_per_cell=cell, orientations=9)
    axs[1, i].imshow(vis, cmap='inferno')
    axs[1, i].set_title(f'cell={cell}, bins=9\n({len(vec)} features)')
    axs[1, i].axis('off')

# Fila 3: Combinaciones
combos = [(8, 12), (16, 9), (32, 6)]
for i, (cell_size, bins) in enumerate(combos):
    vec, vis = descriptor_hog(img, pixels_per_cell=(cell_size, cell_size), orientations=bins)
    axs[2, i].imshow(vis, cmap='inferno')
    axs[2, i].set_title(f'cell=({cell_size},{cell_size}), bins={bins}\n({len(vec)} features)')
    axs[2, i].axis('off')

plt.tight_layout()
plt.savefig('../results/02_hog_variations.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Observaciones:")
print("   - Celdas más pequeñas → más detalle pero más características")
print("   - Más bins de orientación → mejor resolución angular")
print("   - Selección recomendada: cell=(16,16), bins=9 (balance detalle/eficiencia)")


## 3. Momentos de Hu

Los 7 momentos de Hu son **invariantes** a:
- Traslación
- Rotación
- Escala

Referencia: [OpenCV Shape Descriptor: Hu Moments](https://pyimagesearch.com/2014/10/27/opencv-shape-descriptor-hu-moments-example/)


In [ ]:
# Calcular momentos de Hu
hu_moments = descriptor_hu(img)

print("7 Momentos de Hu (transformación logarítmica):")
print("-" * 50)
for i, hu in enumerate(hu_moments):
    print(f"  Hu[{i+1}]: {hu:.6f}")

print(f"\nVector shape: {hu_moments.shape}")

# Visualizar como barras
plt.figure(figsize=(10, 4))
plt.bar(range(1, 8), hu_moments, color='steelblue', edgecolor='black')
plt.xlabel('Momento de Hu')
plt.ylabel('Valor (log)')
plt.title('Los 7 Momentos de Hu')
plt.xticks(range(1, 8))
plt.grid(axis='y', alpha=0.3)
plt.savefig('../results/02_hu_moments.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Demostrar invarianzas de los momentos de Hu
print("Prueba de Invarianzas de los Momentos de Hu")
print("=" * 60)

results = test_hu_invariances(img)

print("\nOriginal:")
print(f"  {results['original']}")

print("\nTraslación (+50, +50):")
print(f"  {results['traslacion']}")

print("\nRotación 90°:")
print(f"  {results['rotacion_90']}")

print("\nEscalado 0.5x:")
print(f"  {results['escalado_05x']}")

# Visualizar las transformaciones
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Original
axes[0].imshow(img, cmap='gray')
axes[0].set_title(f'Original\nHu[1]={results["original"][0]:.4f}')
axes[0].axis('off')

# Traslación
M = np.float32([[1, 0, 50], [0, 1, 50]])
img_shift = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
axes[1].imshow(img_shift, cmap='gray')
axes[1].set_title(f'Traslación\nHu[1]={results["traslacion"][0]:.4f}')
axes[1].axis('off')

# Rotación
M = cv2.getRotationMatrix2D((img.shape[1]//2, img.shape[0]//2), 90, 1)
img_rot = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
axes[2].imshow(img_rot, cmap='gray')
axes[2].set_title(f'Rotación 90°\nHu[1]={results["rotacion_90"][0]:.4f}')
axes[2].axis('off')

# Escalado
img_scale = cv2.resize(img, None, fx=0.5, fy=0.5)
axes[3].imshow(img_scale, cmap='gray')
axes[3].set_title(f'Escalado 0.5x\nHu[1]={results["escalado_05x"][0]:.4f}')
axes[3].axis('off')

plt.suptitle('Demostración de Invarianzas de Momentos de Hu', fontsize=14, y=1.05)
plt.tight_layout()
plt.savefig('../results/02_hu_invariances.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Los valores de Hu son muy similares a pesar de las transformaciones,")
print("   demostrando su invarianza a traslación, rotación y escala.")


## 5. Descriptores de Fourier del Contorno

Los descriptores de Fourier representan el contorno en el dominio de la frecuencia. Los primeros N coeficientes capturan la forma general, mientras que los coeficientes más altos capturan detalles finos.


In [ ]:
# Calcular descriptores de Fourier
n_coeff = 20
fd = descriptor_fourier(img, n_coeff=n_coeff)

print(f"Descriptores de Fourier (primeros {n_coeff} coeficientes):")
print(f"Shape: {fd.shape}")
print(f"Tipo: {fd.dtype}")

# Visualizar magnitudes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(np.abs(fd), 'b-o', markersize=8)
axes[0].set_xlabel('Índice del coeficiente')
axes[0].set_ylabel('Magnitud')
axes[0].set_title('Magnitud de Descriptores de Fourier')
axes[0].grid(True, alpha=0.3)

# Comparar diferentes N
for n in [5, 10, 20, 50]:
    fd_n = descriptor_fourier(img, n_coeff=n)
    axes[1].plot(np.abs(fd_n), label=f'N={n}', marker='o', markersize=4)

axes[1].set_xlabel('Índice del coeficiente')
axes[1].set_ylabel('Magnitud')
axes[1].set_title('Comparación de diferentes N')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/02_fourier_descriptors.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Los primeros coeficientes capturan la forma general del contorno.")
print("   N=20 es un buen balance entre información y dimensionalidad.")


---
# B. DESCRIPTORES DE TEXTURA

## 6. LBP (Local Binary Patterns)

LBP compara cada píxel con sus vecinos para generar un patrón binario. Es robusto a cambios de iluminación.

### Parámetros:
- **P**: Número de puntos vecinos
- **R**: Radio del círculo de vecinos


In [ ]:
# Calcular LBP con parámetros por defecto
lbp_img, lbp_hist = descriptor_lbp(img, P=8, R=1)

print(f"Imagen LBP shape: {lbp_img.shape}")
print(f"Histograma shape: {lbp_hist.shape}")
print(f"Número de bins (patrones): {len(lbp_hist)}")

# Visualizar
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(img, cmap='gray')
axes[0].set_title('Imagen Original')
axes[0].axis('off')

axes[1].imshow(lbp_img, cmap='inferno')
axes[1].set_title('Imagen LBP (P=8, R=1)')
axes[1].axis('off')

axes[2].bar(np.arange(len(lbp_hist)), lbp_hist, color='steelblue')
axes[2].set_xlabel('Patrón LBP')
axes[2].set_ylabel('Frecuencia')
axes[2].set_title('Histograma LBP')

plt.tight_layout()
plt.savefig('../results/02_lbp_basic.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Explorar variaciones de parámetros LBP
configs = [(8, 1), (16, 2), (24, 3)]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for i, (P, R) in enumerate(configs):
    lbp, hist = descriptor_lbp(img, P=P, R=R)
    
    # Imagen LBP
    axes[0, i].imshow(lbp, cmap='inferno')
    axes[0, i].set_title(f'LBP Image (P={P}, R={R})')
    axes[0, i].axis('off')
    
    # Histograma
    axes[1, i].bar(np.arange(len(hist)), hist, color='coral')
    axes[1, i].set_xlabel('Patrón LBP')
    axes[1, i].set_ylabel('Frecuencia')
    axes[1, i].set_title(f'Histograma (P={P}, R={R})\n{len(hist)} bins')

plt.suptitle('Variaciones de Parámetros LBP', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('../results/02_lbp_variations.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Observaciones:")
print("   - Mayor P → más patrones posibles → histogramas más detallados")
print("   - Mayor R → captura textura a mayor escala")
print("   - P=8, R=2 es una buena configuración para radiografías")


## 7. GLCM (Gray Level Co-occurrence Matrix)

GLCM analiza la frecuencia con que pares de píxeles con valores específicos ocurren en la imagen.

### Propiedades calculadas:
- **Contraste**: Medida de la variación local de intensidad
- **Correlación**: Correlación entre píxeles vecinos
- **Energía**: Uniformidad de la distribución
- **Homogeneidad**: Cercanía de valores al diagonal de GLCM


In [ ]:
# Calcular GLCM
img_u8 = (img * 255).astype(np.uint8)
glcm_feats = descriptor_glcm(img_u8)

props = ['contrast', 'correlation', 'energy', 'homogeneity']
print("Descriptores GLCM:")
print("-" * 40)
for prop, value in zip(props, glcm_feats):
    print(f"  {prop}: {value:.6f}")

# Visualizar propiedades
plt.figure(figsize=(10, 5))
plt.bar(props, glcm_feats, color=['#3498db', '#e74c3c', '#2ecc71', '#9b59b6'])
plt.ylabel('Valor')
plt.title('Propiedades GLCM')
plt.xticks(rotation=0)
for i, v in enumerate(glcm_feats):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center', fontsize=10)
plt.tight_layout()
plt.savefig('../results/02_glcm.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Explorar diferentes distancias y ángulos en GLCM
from skimage.feature import graycomatrix, graycoprops

distances = [1, 3, 5]
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
angle_names = ['0°', '45°', '90°', '135°']

results = {}
for d in distances:
    glcm = graycomatrix(img_u8, distances=[d], angles=angles, levels=256, symmetric=True, normed=True)
    for prop in props:
        vals = graycoprops(glcm, prop)
        if prop not in results:
            results[prop] = {}
        results[prop][d] = vals.flatten()

# Visualizar
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for idx, prop in enumerate(props):
    ax = axes[idx // 2, idx % 2]
    x = np.arange(len(angle_names))
    width = 0.25
    
    for i, d in enumerate(distances):
        ax.bar(x + i*width, results[prop][d], width, label=f'd={d}')
    
    ax.set_xlabel('Ángulo')
    ax.set_ylabel(prop.capitalize())
    ax.set_title(f'GLCM: {prop.capitalize()}')
    ax.set_xticks(x + width)
    ax.set_xticklabels(angle_names)
    ax.legend()

plt.suptitle('Variación de propiedades GLCM con distancia y ángulo', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('../results/02_glcm_variations.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Las propiedades GLCM varían según la distancia y dirección analizada.")
print("   Se recomienda promediar sobre múltiples ángulos para mayor robustez.")


## 8. Filtros de Gabor

Los filtros de Gabor capturan información de textura a diferentes frecuencias y orientaciones, similar al sistema visual humano.


In [ ]:
# Calcular descriptores de Gabor
gabor_feats = descriptor_gabor(img)

print(f"Vector de características Gabor: {len(gabor_feats)} valores")
print(f"(3 frecuencias × 3 orientaciones × 2 estadísticas = 18 features)")

# Visualizar respuestas de Gabor
from skimage.filters import gabor

freqs = [0.1, 0.2, 0.3]
thetas = [0, np.pi/4, np.pi/2]
theta_names = ['0°', '45°', '90°']

fig, axes = plt.subplots(3, 3, figsize=(12, 12))

for i, freq in enumerate(freqs):
    for j, theta in enumerate(thetas):
        real, _ = gabor(img, frequency=freq, theta=theta)
        axes[i, j].imshow(real, cmap='gray')
        axes[i, j].set_title(f'f={freq}, θ={theta_names[j]}')
        axes[i, j].axis('off')

plt.suptitle('Respuestas de Filtros de Gabor', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('../results/02_gabor.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Cada filtro de Gabor es sensible a texturas en una frecuencia y orientación específica.")
print("   Útil para capturar patrones direccionales en las radiografías.")


## 9. Estadísticas de Primer Orden

Métricas estadísticas básicas sobre la distribución de intensidades:
- **Media**: Valor promedio de intensidad
- **Varianza**: Dispersión de valores
- **Skewness**: Asimetría de la distribución
- **Kurtosis**: "Peakedness" de la distribución
- **Entropía**: Medida de aleatoriedad/información


In [ ]:
# Calcular estadísticas de primer orden
stats = first_order_stats(img)

print("Estadísticas de Primer Orden:")
print("-" * 40)
for key, value in stats.items():
    print(f"  {key}: {value:.6f}")

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de la imagen
axes[0].hist(img.flatten(), bins=256, color='steelblue', alpha=0.7)
axes[0].axvline(stats['mean'], color='red', linestyle='--', label=f"Media: {stats['mean']:.3f}")
axes[0].set_xlabel('Intensidad')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de Intensidades')
axes[0].legend()

# Barras de estadísticas
stat_names = list(stats.keys())
stat_values = list(stats.values())
colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6', '#f39c12']
axes[1].bar(stat_names, stat_values, color=colors)
axes[1].set_ylabel('Valor')
axes[1].set_title('Estadísticas de Primer Orden')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../results/02_first_order_stats.png', dpi=150, bbox_inches='tight')
plt.show()


---
## 10. Resumen de Descriptores Implementados

### A. Descriptores de Forma
| Descriptor | Dimensionalidad | Características |
|------------|-----------------|-----------------|
| HOG | ~1764 (16×16 cell) | Gradientes orientados, detecta bordes |
| Momentos de Hu | 7 | Invariantes a traslación, rotación, escala |
| Contorno | 4 | Área, perímetro, circularidad, excentricidad |
| Fourier | N (ej: 20) | Representación frecuencial del contorno |

### B. Descriptores de Textura
| Descriptor | Dimensionalidad | Características |
|------------|-----------------|-----------------|
| LBP | P+2 (ej: 10) | Patrones locales, robusto a iluminación |
| GLCM | 4 | Contraste, correlación, energía, homogeneidad |
| Gabor | 18 (3×3×2) | Texturas multi-frecuencia y orientación |
| Estadísticas | 5 | Media, varianza, skewness, kurtosis, entropía |


In [ ]:
# Resumen de dimensionalidad total
from src.features import extract_features_all

# Extraer todas las características de la imagen de muestra
all_features = extract_features_all(img)

print("="*60)
print("RESUMEN DE EXTRACCIÓN DE CARACTERÍSTICAS")
print("="*60)
print(f"\nVector de características total: {len(all_features)} dimensiones")
print(f"Tipo de datos: {all_features.dtype}")
print(f"Rango de valores: [{all_features.min():.4f}, {all_features.max():.4f}]")

# Desglose aproximado
print("\nDesglose aproximado:")
print(f"  - HOG:         ~1764 features (depende de parámetros)")
print(f"  - Hu Moments:      7 features")
print(f"  - Fourier:        20 features (magnitudes)")
print(f"  - LBP:            10 features (histograma)")
print(f"  - GLCM:            4 features")
print(f"  - Gabor:          18 features")
print(f"  - Total:       ~1823 features")
print("="*60)


## 11. Conclusiones

### Descriptores de Forma implementados:
1. ✅ **HOG**: Captura gradientes orientados, útil para detectar estructuras anatómicas
2. ✅ **Momentos de Hu**: 7 valores invariantes a transformaciones geométricas
3. ✅ **Descriptores de Contorno**: Métricas geométricas del contorno segmentado
4. ✅ **Fourier**: Representación frecuencial del contorno

### Descriptores de Textura implementados:
1. ✅ **LBP**: Patrones binarios locales, robustos a cambios de iluminación
2. ✅ **GLCM**: Propiedades de co-ocurrencia de niveles de gris
3. ✅ **Gabor**: Filtros multi-frecuencia y multi-orientación
4. ✅ **Estadísticas de primer orden**: Métricas básicas de distribución

### Próximos pasos:
- **Notebook 03**: Usar estos descriptores para clasificar NORMAL vs PNEUMONIA
- Comparar rendimiento de clasificadores clásicos vs CNN


In [ ]:
# Calcular descriptores de contorno
contorno_feats = descriptor_contorno(img)

print("Descriptores de Contorno:")
print("-" * 40)
for key, value in contorno_feats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

# Visualizar segmentación y contorno
mask = segmentar(img)
cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

if len(cnts) > 0:
    cnt = max(cnts, key=cv2.contourArea)
    
    # Crear imagen para dibujar
    img_contour = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR)
    cv2.drawContours(img_contour, [cnt], -1, (0, 255, 0), 2)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title('Imagen Preprocesada')
    axes[0].axis('off')
    
    axes[1].imshow(mask, cmap='gray')
    axes[1].set_title('Máscara (Segmentación Otsu)')
    axes[1].axis('off')
    
    axes[2].imshow(cv2.cvtColor(img_contour, cv2.COLOR_BGR2RGB))
    axes[2].set_title(f'Contorno Detectado\nÁrea={contorno_feats["area"]:.0f}, Circ={contorno_feats["circularidad"]:.3f}')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.savefig('../results/02_contour_descriptors.png', dpi=150, bbox_inches='tight')
    plt.show()
